# Demo de split simple

# 1. Generar los datasets para entrenar YOLO


In [2]:
import os
import shutil
import random

def split_dataset_yolo(
    images_dir: str,
    labels_dir: str,
    train_images_dir: str,
    train_labels_dir: str,
    val_images_dir: str,
    val_labels_dir: str,
    train_ratio: float = 0.8,
    seed: int = 42
):
    """
    Divide un conjunto de datos tipo YOLO en entrenamiento y validación.

    Args:
        images_dir (str): Ruta de imágenes originales.
        labels_dir (str): Ruta de etiquetas originales.
        train_images_dir (str): Carpeta destino para imágenes de entrenamiento.
        train_labels_dir (str): Carpeta destino para etiquetas de entrenamiento.
        val_images_dir (str): Carpeta destino para imágenes de validación.
        val_labels_dir (str): Carpeta destino para etiquetas de validación.
        train_ratio (float): Porcentaje para entrenamiento (entre 0 y 1).
        seed (int): Semilla para reproducibilidad.
    """
    # Preparar carpetas
    for path in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
        if os.path.exists(path):
            shutil.rmtree(path)
        os.makedirs(path, exist_ok=True)

    # Obtener lista de imágenes
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Dividir aleatoriamente
    random.seed(seed)
    random.shuffle(image_files)
    split_idx = int(train_ratio * len(image_files))
    train_files = image_files[:split_idx]
    val_files = image_files[split_idx:]

    # Función para copiar imágenes y etiquetas
    def copy_files(file_list, dst_img_dir, dst_lbl_dir, set_name):
        for fname in file_list:
            shutil.copy2(os.path.join(images_dir, fname), os.path.join(dst_img_dir, fname))
            label_file = os.path.splitext(fname)[0] + '.txt'
            label_path = os.path.join(labels_dir, label_file)
            if os.path.exists(label_path):
                shutil.copy2(label_path, os.path.join(dst_lbl_dir, label_file))
            else:
                print(f"[⚠️] Etiqueta no encontrada para {fname}")
        print(f"✅ {set_name}: {len(file_list)} imágenes copiadas.")

    # Copiar a cada carpeta
    copy_files(train_files, train_images_dir, train_labels_dir, "Entrenamiento")
    copy_files(val_files, val_images_dir, val_labels_dir, "Validación")
    print("🚀 División completada con éxito.")

In [3]:
# Votación
split_dataset_yolo(
    images_dir="../data/dataset_etiquetado_zonas/d_etiquetados/images",
    labels_dir="../data/dataset_etiquetado_zonas/d_etiquetados/labels",

    train_images_dir="../data/dataset_etiquetado_zonas/e_split/train/images",
    train_labels_dir="../data/dataset_etiquetado_zonas/e_split/train/labels",
    val_images_dir="../data/dataset_etiquetado_zonas/e_split/val/images",
    val_labels_dir="../data/dataset_etiquetado_zonas/e_split/val/labels",
    train_ratio=0.8
)

✅ Entrenamiento: 240 imágenes copiadas.
✅ Validación: 60 imágenes copiadas.
🚀 División completada con éxito.


# 2. Generar los YML - Copiar archivos


In [4]:
import os
import shutil

def copiar_archivos_generales(origen_dir: str, destino_dir: str, archivos: list[str]):
    """
    Copia archivos generales como 'classes.txt' o 'notes.json' desde origen a destino.

    Args:
        origen_dir (str): Ruta de origen donde están los archivos.
        destino_dir (str): Ruta de destino.
        archivos (list): Lista de nombres de archivos a copiar.
    """
    for file_name in archivos:
        src = os.path.join(origen_dir, file_name)
        dst = os.path.join(destino_dir, file_name)
        if os.path.exists(src):
            shutil.copy2(src, dst)
            print(f"✅ Copiado '{file_name}' a '{dst}'")
        else:
            print(f"⚠️ [ADVERTENCIA] '{file_name}' no encontrado en '{origen_dir}'")

In [5]:
def generar_dataset_yaml(classes_path: str, dataset_dir: str, output_file: str = "dataset.yml"):
    """
    Genera el archivo dataset.yml requerido por YOLO con rutas absolutas.

    Args:
        classes_path (str): Ruta al archivo classes.txt.
        dataset_dir (str): Directorio raíz donde están las carpetas 'train' y 'val'.
        output_file (str): Nombre del archivo de salida YAML (por defecto 'dataset.yml').
    """
    if not os.path.exists(classes_path):
        print(f"⚠️ [ADVERTENCIA] No se encontró '{classes_path}', no se generó dataset.yml.")
        return

    with open(classes_path, "r", encoding="utf-8") as f:
        class_names = [line.strip() for line in f if line.strip()]

    names_yaml = "\n".join([f"  - {name}" for name in class_names])

    content = f"""train: {os.path.abspath(os.path.join(dataset_dir, "train"))}
val: {os.path.abspath(os.path.join(dataset_dir, "val"))}

nc: {len(class_names)}
names:
{names_yaml}
"""

    yaml_path = os.path.join(dataset_dir, output_file)
    with open(yaml_path, "w", encoding="utf-8") as f:
        f.write(content)

    print(f"✅ Archivo '{output_file}' creado en '{yaml_path}'")

In [ ]:
# Votación
copiar_archivos_generales(
    origen_dir="../data/dataset_etiquetado_zonas/d_etiquetados",
    destino_dir="../data/dataset_etiquetado_zonas/e_split",
    archivos=["classes.txt", "notes.json"]
)

# Votacion
generar_dataset_yaml(
    classes_path="../data/dataset_etiquetado_zonas/e_split/classes.txt",
    dataset_dir="../data/dataset_etiquetado_zonas/e_split"
)

✅ Copiado 'classes.txt' a '../data/dataset_etiquetado_zonas/e_split/classes.txt'
✅ Copiado 'notes.json' a '../data/dataset_etiquetado_zonas/e_split/notes.json'
✅ Archivo 'dataset.yml' creado en '../data/dataset_etiquetado_zonas/e_split/dataset.yml'
